# Nodejs Tensorflow Example

 * Wrap a nodejs tensorflow model for use as a prediction microservice in seldon-core
   * Run locally on Docker to test
 
## Dependencies

 * ```pip install seldon-core```
 * [Helm](https://github.com/kubernetes/helm)
 * [Minikube](https://github.com/kubernetes/minikube)
 * [S2I](https://github.com/openshift/source-to-image)
 * node (version>=8.11.0)
 * npm

## Train locally using npm commands
This model example takes an input of 10 different features and predicts a out for the same. For the training part it uses a random normally distributed input set of 100 rows i.e a data set of [100,10] and trains it for another random normally distributed data set of size [100,1]. For every prediction the model expects a dataset of dimension [r,10] where r is the num of input rows to be predicted.

In [1]:
!make train && make clean_build

npm install
        ......] \ refresh-package-json:@tensorflow/tfjs-node: timing actiom
> @tensorflow/tfjs-node@0.1.15 install /home/clive/work/seldon-core/fork-seldon-core/examples/models/nodejs_tensorflow/node_modules/@tensorflow/tfjs-node
> node scripts/install.js

* Downloading libtensorflow
[                              ] Infinity/bps 0% 0.0s[                              ] 154364/bps 0% 128.0s[                              ] 1240158/bps 0% 15.9s[                              ] 3603148/bps 0% 5.4s[=                             ] 4762971/bps 1% 4.1s[=                             ] 5481686/bps 2% 3.5s[=                             ] 6214471/bps 3% 3.1s[=                             ] 6967373/bps 4% 2.7s[=                             ] 7131544/bps 4% 2.6s[==                            ] 6972763/bps 5% 2.7s[==                            ] 9257518/bps 7% 2.0s[===                           ] 11301908/bps 10% 1.6s[====                          ] 12855355/bps 13% 1.3s[=====              

Epoch 95: loss = 0.5841403007507324
Epoch 96: loss = 0.5787111520767212
Epoch 97: loss = 0.5767002105712891
Epoch 98: loss = 0.5712048411369324
Epoch 99: loss = 0.5715557932853699
/home/clive/work/seldon-core/fork-seldon-core/examples/models/nodejs_tensorflow


   ╭───────────────────────────────────────────────────────────────╮
   │                                                               │
   │       New minor version of npm available! 6.4.1 → 6.9.0       │
   │   Changelog: https://github.com/npm/cli/releases/tag/v6.9.0   │
   │               Run npm install -g npm to update!               │
   │                                                               │
   ╰───────────────────────────────────────────────────────────────╯

rm -rf node_modules
rm -f package-lock.json


Training creates a model.json file and a weights.bin file which is utilized for prediction

## Prediction using REST API on the docker container

In [2]:
!s2i build . seldonio/seldon-core-s2i-nodejs:0.2-SNAPSHOT node-s2i-model-image:0.1

---> Installing application source...
---> Installing dependencies ...

> @tensorflow/tfjs-node@0.1.15 install /microservice/model/node_modules/@tensorflow/tfjs-node
> node scripts/install.js

* Downloading libtensorflow

* Building TensorFlow Node.js bindings

> protobufjs@6.8.8 postinstall /microservice/model/node_modules/protobufjs
> node scripts/postinstall

npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN nodejs_tensorflow@1.0.0 No repository field.
npm WARN nodejs_tensorflow@1.0.0 No license field.

added 48 packages from 56 contributors and audited 61 packages in 8.775s
found 0 vulnerabilities

Build completed successfully


In [3]:
!docker run --name "nodejs_tensorflow_predictor" -d --rm -p 5000:5000 node-s2i-model-image:0.1

6cc8e4bca5aff59b1a4f0613f4e61ac212bd513954f4c61c964c0cb237a35f34


Send some random features that conform to the contract

In [4]:
!seldon-core-tester contract.json 0.0.0.0 5000 -p -t

----------------------------------------
SENDING NEW REQUEST:

[[5.742 7.559 0.876 8.28  0.631 5.414 0.392 0.822 8.55  9.548]]
RECEIVED RESPONSE:
data {
  names: "t:0"
  tensor {
    shape: 1
    shape: 1
    values: -1.732214331626892
  }
}




In [5]:
!docker rm nodejs_tensorflow_predictor --force

nodejs_tensorflow_predictor


## Prediction using GRPC API on the docker container

In [6]:
!s2i build -E ./.s2i/environment_grpc . seldonio/seldon-core-s2i-nodejs:0.2-SNAPSHOT node-s2i-model-image:0.2

---> Installing application source...
---> Installing dependencies ...

> @tensorflow/tfjs-node@0.1.15 install /microservice/model/node_modules/@tensorflow/tfjs-node
> node scripts/install.js

* Downloading libtensorflow

* Building TensorFlow Node.js bindings

> protobufjs@6.8.8 postinstall /microservice/model/node_modules/protobufjs
> node scripts/postinstall

npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN nodejs_tensorflow@1.0.0 No repository field.
npm WARN nodejs_tensorflow@1.0.0 No license field.

added 48 packages from 56 contributors and audited 61 packages in 7.096s
found 0 vulnerabilities

Build completed successfully


In [7]:
!docker run --name "nodejs_tensorflow_predictor" -d --rm -p 5000:5000 node-s2i-model-image:0.2

3e824d60a31f688ed8c5e7cc95cb6e15ff72669faec8e219d6fee3a900794007


Send some random features that conform to the contract

In [8]:
!seldon-core-tester contract.json 0.0.0.0 5000 -p -t --grpc

----------------------------------------
SENDING NEW REQUEST:

[[8.196e+00 9.259e+00 8.349e+00 9.000e-03 5.450e+00 5.363e+00 2.453e+00
  3.760e-01 4.719e+00 7.410e-01]]
RECEIVED RESPONSE:
data {
  names: "t:0"
  tensor {
    shape: 1
    shape: 1
    values: 3.598963499069214
  }
}




In [9]:
!docker rm nodejs_tensorflow_predictor --force

nodejs_tensorflow_predictor


## Test using Minikube

**Due to a [minikube/s2i issue](https://github.com/SeldonIO/seldon-core/issues/253) you will need [s2i >= 1.1.13](https://github.com/openshift/source-to-image/releases/tag/v1.1.13)**

In [ ]:
!minikube start --memory 4096

## Setup Seldon Core

Use the setup notebook to [Setup Cluster](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Setup-Cluster) with [Ambassador Ingress](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Ambassador) and [Install Seldon Core](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Install-Seldon-Core). Instructions [also online](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html).

## Build image and test

In [17]:
!eval $(minikube docker-env) && s2i build . seldonio/seldon-core-s2i-nodejs:0.2-SNAPSHOT node-s2i-model-image:0.1

---> Installing application source...
---> Installing dependencies ...

> @tensorflow/tfjs-node@0.1.15 install /microservice/model/node_modules/@tensorflow/tfjs-node
> node scripts/install.js

* Downloading libtensorflow

* Building TensorFlow Node.js bindings

> protobufjs@6.8.8 postinstall /microservice/model/node_modules/protobufjs
> node scripts/postinstall

npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN nodejs_tensorflow@1.0.0 No repository field.
npm WARN nodejs_tensorflow@1.0.0 No license field.

added 48 packages from 56 contributors and audited 61 packages in 10.745s
found 0 vulnerabilities

Build completed successfully


In [18]:
!kubectl create -f nodejs_tensorflow_deployment.json

seldondeployment.machinelearning.seldon.io/seldon-deployment-example created


In [19]:
!kubectl rollout status deploy/seldon-cea8a97ce503f62508ad289c86fe0e27

Waiting for deployment "seldon-cea8a97ce503f62508ad289c86fe0e27" rollout to finish: 0 of 1 updated replicas are available...
deployment "seldon-cea8a97ce503f62508ad289c86fe0e27" successfully rolled out


In [20]:
!seldon-core-api-tester contract.json `minikube ip` `kubectl get svc ambassador -o jsonpath='{.spec.ports[0].nodePort}'` \
    seldon-deployment-example --namespace default -p

----------------------------------------
SENDING NEW REQUEST:

[[8.804 4.565 8.703 3.414 6.527 2.084 4.716 0.56  0.08  6.324]]
RECEIVED RESPONSE:
meta {
  puid: "37h6oijvi0g2iu10hlvgcnojms"
  requestPath {
    key: "nodejs-tensorflow-predictor"
    value: "node-s2i-model-image:0.1"
  }
}
data {
  names: "t:0"
  ndarray {
    values {
      list_value {
        values {
          number_value: -0.05151659995317459
        }
      }
    }
  }
}




In [ ]:
!minikube delete

In [21]:
!make clean

rm -rf node_modules
rm -f package-lock.json
rm -f model.json
rm -f weights.bin
